# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p><b>1. What test is appropriate for this problem? Does CLT apply?</b>
<br>
We will use a permutation test. We are testing whether a qualitative factor makes a difference for a variable in the data set, so the statistic in question is the difference of two means.
<br>
<br>
The Central Limit theorem applies because enough data points have been collected, so we can assume independence.
<br>
<br>
<b>2. What are the null and alternate hypotheses?</b>
<br>
The null hypothesis is that there is no difference between callbacks for black and white. In other words, the mean callbacks for white-sounding names is the same as the mean callbacks for black-sounding names, so when we take their difference, it should equal zero (or be reasonably close to zero). The alternative hypothesis is that there is any difference, one way or the other. We will use a two-tailed test and set significance/condifence to 95%.
</p>
</div>

In [6]:
w = data[data.race=='w']
b = data[data.race=='b']

In [20]:
# Find the empirically observed statistic.

emp_diff = np.mean(w.call) - np.mean(b.call)

print('Proportion of white names that get callbacks:', np.mean(w.call))
print('Proportion of white names that get callbacks:', np.mean(b.call))
print('Observed difference in means:', emp_diff)

# We observe white-sounding names are about 50% more likely to receive callbacks,
# beating black names by three full percentage points.

def draw_perm_reps(data_1, data_2, func, size=1):
    
    # Initialize an empty array of replicates
    perm_replicates = np.empty(size)
    
    data = np.concatenate((data_1, data_2))
    
    for i in range(size):
        
        # Scramble the data
        data = np.random.permutation(data)
        
        # Generate a permuation
        perm_1, perm_2 = data[:len(data_1)], data[len(data_1):]
        
        # Compute the test statistic
        perm_replicates[i] = func(perm_1, perm_2)
        
    return perm_replicates

def diff_of_means(data_1, data_2):
    
    return np.mean(data_1) - np.mean(data_2)

# Our permutation test will scramble the callbacks into two piles,
# then see what the difference between those random piles is.
# It should nearly always be close to 0%, but we are interested
# in how many times it is 3% or higher.

race_perm_test = draw_perm_reps(w.call, b.call, diff_of_means, 10000)

p = np.sum(race_perm_test >= emp_diff) / len(race_perm_test)

print('The p-value is:', p)

# This p-value is far lower than our signifance threshold of .05; we reject the null hypothesis.
# The empirically observed difference is very unlikely to have resulted from random chance.

Proportion of white names that get callbacks: 0.09650924056768417
Proportion of white names that get callbacks: 0.0644763857126236
Observed difference in means: 0.03203285485506058
The p-value is: 0.0001


<div class="span5 alert alert-success">
<p><b>4. Write a story describing the statistical significance in the context or the original problem.</b>
<br>
Think of the initial experiment as sorting the data in to 'black-sounding' and 'white-sounding'. For both groups, we can compute the callback mean (i.e. the proportion of candidates which received a callback), then we can see what the difference between the two groups' means is. We observed a difference of 3%. To test whether that difference could have happened by chance, we scrambled the callback data into two meaningless groups of the same size as our original groups, and compute the difference of those random groups' means—it should be close to zero. We did that ten thousand times. That allowed us to see how often we get a 3% difference across two random groups, compared to when the data were divided into two meaningful groups (i.e. 'black' and 'white'). We found that we <i>practically never</i> see a 3% difference or higher due to chance, so we can conclude with confidence that we observed an outcome difference between black-sounding names and white-sounding names. White-sounding names get about 50% more callbacks (9.6% of the time instead of 6.4% of the time).
<br>
<br>
<b>5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?</b>
<br>
Not at all! My analysis merely prdemonstrates that race/name is <b>one</b> of the factors in callback success. I cannot make any claims about the other variables, because I did not test them. In order to determine which factor is most important, I would need to perform more tests on each of the other variables presented. I would want to see which factor correlates most strongly with success, not just which one is the most statistically significant. Multivariable statistics would show which factors are most correlated with success, or we could create a machine learning model which determines the most important factors by trial and error.
</p>
</div>